In [1]:
import csv
import pandas as pd

In [2]:
def read_csv(file_path):
    df = pd.read_csv(file_path)
    return df


In [3]:
vL = read_csv('SPvLHH.csv')
#data pulled from Savant search
    #batter handedness = left, pos = SP, team = tigers, min pitches = 25, group by player & pitch type
    #https://baseballsavant.mlb.com/statcast_search?hfPT=&hfAB=&hfGT=R%7C&hfPR=&hfZ=&hfStadium=&hfBBL=&hfNewZones=&hfPull=&hfC=&hfSea=2025%7C&hfSit=&player_type=pitcher&hfOuts=&hfOpponent=&pitcher_throws=&batter_stands=L&hfSA=&game_date_gt=&game_date_lt=&hfMo=&hfTeam=116%7C&home_road=&hfRO=&position=SP&hfInfield=&hfOutfield=&hfInn=&hfBBT=&hfFlag=&metric_1=&group_by=pitch-type&min_pitches=0&min_results=25&min_pas=0&sort_col=pitches&player_event_sort=api_p_release_speed&sort_order=desc&chk_stats_release_pos_z=on&chk_stats_release_pos_x=on&chk_stats_release_extension=on&chk_stats_arm_angle=on#results

vL.player_name = vL.player_name.str.split(', ').map(lambda x : ' '.join(x[::-1]))
#prefer firstname lastname to lastname, firstname
vL['batter_hand'] = 'L'

vR = read_csv('SPvRHH.csv')
vR.player_name = vR.player_name.str.split(', ').map(lambda x : ' '.join(x[::-1]))
vR['batter_hand']='R'

In [4]:
combined_pitch = pd.concat([vR, vL],ignore_index=True)


#metrics = ['pitches','release_extension','release_pos_x','release_pos_z','arm_angle']
##initially included to use values=metrics below, but I don't want to use pitches for my difference column later

diff = combined_pitch.pivot(index=['player_name', 'pitch_type'],columns='batter_hand',
    values=['pitches','release_extension','release_pos_x','release_pos_z','arm_angle'])
#

diff_clean = diff.dropna().copy()
#removes rows where pitchers did not throw 25 pitches per type to both handed batters
##initially ran into SettingWithCopyWarning, had to add the .copy()

for metric in ['release_extension','release_pos_x','release_pos_z','arm_angle']:
    diff_clean.loc[:, (metric, 'Difference (L-R)')] = diff_clean[metric]['L'] - diff_clean[metric]['R']

In [5]:
diff

pitches        release_extension        \
batter_hand                         L      R                 L     R   
player_name        pitch_type                                          
Brant Hurter       SI            40.0   34.0              6.20  6.14   
Casey Mize         FF           519.0  282.0              6.91  6.94   
                   FS           408.0  161.0              6.85  6.93   
                   SI             NaN  275.0               NaN  6.92   
                   SL           233.0  127.0              6.88  6.98   
                   SV           106.0  230.0              6.85  6.89   
Charlie Morton     CH            57.0    NaN              6.26   NaN   
                   CU           167.0  109.0              6.09  6.08   
                   FC            30.0   54.0              6.13  6.19   
                   FF           108.0   62.0              6.14  6.17   
                   SI             NaN  108.0               NaN  6.18   
Chris Paddack      CH            97.0    NaN              6.96   NaN   
                   CU            41.0   32.0              6.69  6.73   
                   FC            32.0   26.0              6.90  6.91   
                   FF           128.0   94.0              7.00  6.98   
                   SI             NaN   37.0               NaN  6.97   
Dietrich Enns      CH             NaN   34.0               NaN  6.33   
                   FF             NaN   49.0               NaN  6.34   
Jack Flaherty      CH            61.0    NaN              6.55   NaN   
                   FF           686.0  641.0              6.53  6.60   
                   KC           385.0  333.0              6.45  6.47   
                   SL           366.0  303.0              6.48  6.54   
Jackson Jobe       CH            99.0   49.0              6.02  6.03   
                   CU            50.0    NaN              5.97   NaN   
                   FF           157.0  116.0              5.97  6.02   
                   SI             NaN  115.0               NaN  6.00   
                   SL           119.0  149.0              5.92  6.01   
Keider Montero     CH            89.0    NaN              6.48   NaN   
                   FF           202.0   97.0              6.34  6.35   
                   KC            83.0   39.0              6.20  6.19   
                   SI             NaN  181.0               NaN  6.34   
                   SL            72.0  150.0              6.22  6.26   
Reese Olson        CH           160.0  140.0              5.88  5.87   
                   CU            45.0    NaN              5.77   NaN   
                   FF           169.0   44.0              5.90  5.89   
                   SI            51.0  276.0              5.93  5.92   
                   SL           111.0  139.0              5.84  5.86   
Sawyer Gipson-Long CH            32.0    NaN              7.50   NaN   
                   FF            45.0    NaN              7.36   NaN   
                   SI             NaN   34.0               NaN  7.34   
                   SL            25.0   33.0              7.24  7.31   
Tarik Skubal       CH           103.0  792.0              6.47  6.43   
                   CU             NaN   78.0               NaN  6.46   
                   FF           109.0  732.0              6.49  6.47   
                   SI           222.0  453.0              6.52  6.50   
                   SL            90.0  266.0              6.51  6.49   
Troy Melton        FF            66.0   40.0              6.94  6.92   
                   SI             NaN   38.0               NaN  7.07   
                   SL            29.0   34.0              7.01  6.96   
Tyler Holton       FC             NaN   33.0               NaN  6.50   
                   SI            36.0    NaN              6.46   NaN   

                              release_pos_x       release_pos_z        \
batter_hand                               L     R             L     

In [81]:
diff_clean.to_csv('rel pt diff.csv',index=True)